In [1]:
import itertools
import math
import struct

# Build an image

In [2]:
# Layout: 5 . . 6 7 . 8 . 9
# Total size: 8 * 65536 + 512 = 524800 bytes

In [3]:
data = bytearray(524488)
for pos in itertools.chain(
    range(0, 65536, 4),
    range(3 * 65536, 5 * 65536, 4),
    range(6 * 65536, 7 * 65536, 4),
    range(8 * 65536, 8 * 65536 + 512, 4),
):
    data[pos:pos + 4] = struct.pack('>L', pos)
    
with open('test.raw', 'wb') as out:
    out.write(data)

In [4]:
out = open('test.qcow2', 'wb')

# Image cluster layout:
# 0: header
# 1: refcount table
# 2: refcount block #0
# 3: L1 table
# 4: L2 table #0
# 5: guest cluster #0
# 6: guest cluster #3
# 7: guest cluster #4
# 8: guest cluster #6
# 9: guest cluster #8

# Guest data layout:
# 0: stored in cluster #5
# 1: zero
# 2: zero
# 3: stored in cluster #6
# 4: stored in cluster #7
# 5: zero
# 6: stored in cluster #8
# 7: zero
# 8: stored in cluster #9

#
# HEADER
#

# Magic
out.write(b'QFI\xFB')

# Version
out.write(struct.pack('>L', 2))

# Backing file name offset (0 = no backing file)
out.write(struct.pack('>Q', 0))

# Backing file name length
out.write(struct.pack('>L', 0))

# Number of bits per cluster address, 1<<bits is the cluster size
# Let's use 64KiB clusters (bits=16)
cluster_size = 1 << 16
assert cluster_size == 65536
out.write(struct.pack('>L', 16))

# Virtual disk size in bytes
out.write(struct.pack('>Q', 524800))

assert out.tell() == 32

# Encryption method (none)
out.write(struct.pack('>L', 0))

# L1 table size (number of entries)
out.write(struct.pack('>L', 1))

# L1 table offset
out.write(struct.pack('>Q', 3 * 65536))

# Refcount table offset
out.write(struct.pack('>Q', 1 * 65536))

# Refcount table length in clusters
out.write(struct.pack('>L', 1))

# Number of snapshots in the image
out.write(struct.pack('>L', 0))

# Offset of the snapshot table (must be aligned to clusters)
out.write(struct.pack('>Q', 0))

assert out.tell() == 72

#
# REFCOUNT TABLE
#

out.write(b'\x00' * (1 * 65536 - out.tell()))

# Offset of the only refcount block
out.write(struct.pack('>Q', 2 * 65536))

#
# REFCOUNT BLOCK
#

out.write(b'\x00' * (2 * 65536 - out.tell()))

# Reference counts of all the clusters
for count in [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]:
    out.write(struct.pack('>H', count))

#
# L1 TABLE
#

out.write(b'\x00' * (3 * 65536 - out.tell()))

# Offset of the only L2 table
offset = 4 * 65536
refcount_is_one = 1

l1_entry = offset | (refcount_is_one << 63)
out.write(struct.pack('>Q', l1_entry))

#
# L2 TABLE
#

out.write(b'\x00' * (4 * 65536 - out.tell()))

for entry in [5, None, None, 6, 7, None, 8, None, 9]:
    if entry is None:
        l2_entry = 0
    else:
        offset = entry * 65536
        l2_entry = (
            offset
            | (0 << 62)  # Standard cluster
            | (1 << 63)  # Standard cluster with refcount=1
        )
    out.write(struct.pack('>Q', l2_entry))
    
assert out.tell() < 5 * 65536

# Fill out the rest of the entries with zeros
for _ in range((5 * 65536 - out.tell()) // 8):
    l2_entry = 0
    out.write(struct.pack('>Q', l2_entry))
    
assert out.tell() == 5 * 65536

#
# DATA
#

for offset in [0, 3, 4, 6, 8]:
    d = data[(offset * 65536):((offset + 1) * 65536)]
    d += b'\x00' * (65536 - len(d))
    assert len(d) == 65536, len(d)
    out.write(d)
    
assert out.tell() == 10 * 65536

out.close()